In [2]:
import pandas as pd

In [3]:
dt_execute_rbg = pd.read_pickle("result/Execute RBG.pkl")
dt_start_handle_request = pd.read_pickle("result/Start Handle Request.pkl")
dt_end_handle_request = pd.read_pickle("result/End Handle Request.pkl")
dt_path_movement_finished = pd.read_pickle("result/Path Movement Finished.pkl")
dt_path_movement_finished_detail = pd.read_pickle("result/Path Movement Finished - Detail.pkl")
dt_path_movement_failed = pd.read_pickle("result/Path Movement Failed.pkl")
dt_unknown = pd.read_pickle("result/Unknown Content.pkl")
dt_alerts_handled = pd.read_pickle("result/Alerts Handled.pkl")
dt_alerts_processing = pd.read_pickle("result/Alerts Processing.pkl")
dt_alerts_received = pd.read_pickle("result/Alerts Received.pkl")
dt_telegrams_processed = pd.read_pickle("result/Telegrams Processed.pkl")
dt_telegrams_sent = pd.read_pickle("result/Telegrams Sent.pkl")
dt_is_position_ok = pd.read_pickle("result/Check Position.pkl")
dt_check_sequence = pd.read_pickle("result/Check Sequence.pkl")
dt_is_vb_ok = pd.read_pickle("result/IsVBOK.pkl")


In [4]:
dt_is_position_ok[dt_is_position_ok['timestamp'] == '2025.02.06 11:12:47.801']

,timestamp,module,worker_id,id,position,status
1499,2025.02.06 11:12:47.801,MF3,G42698514,1847754,1118,seq=1716.9520852.0.0[9520852]{3}


In [104]:
dt_path_movement_finished_detail[dt_path_movement_finished_detail['timestamp'] == '2025.02.06 07:00:00.106']

,timestamp,mfs_id,order,from,to
0,2025.02.06 07:00:00.106,1847105,1,103005,103100
1,2025.02.06 07:00:00.106,1847105,2,103100,1310
2,2025.02.06 07:00:00.106,1847105,3,1310,1318
3,2025.02.06 07:00:00.106,1847105,4,1318,1715
4,2025.02.06 07:00:00.106,1847105,5,1715,1716


In [84]:
dt_path_movement_finished[dt_path_movement_finished['status'] == 'false']

,timestamp,module,worker_id,search_status,mfs_id,status,code,paths
189,2025.02.06 07:00:41.720,MF1,G42038421,finished,1847162,false,-2,[]
220,2025.02.06 07:00:48.632,MF1,G42038740,finished,1847162,false,-2,[]
222,2025.02.06 07:00:48.820,MF1,G42038754,finished,1847127,false,-2,[]
257,2025.02.06 07:00:55.742,MF1,G42039090,finished,1847127,false,-2,[]
653,2025.02.06 07:02:22.584,MF1,G42041851,finished,1847145,false,-2,[]
...,...,...,...,...,...,...,...,...
57425,2025.02.06 11:33:38.242,MF1,G42726098,finished,1847722,false,-2,[]
57440,2025.02.06 11:33:44.478,MF1,G42726247,finished,1847708,false,-2,[]
57452,2025.02.06 11:33:49.090,MF1,G42726357,finished,1847734,false,-2,[]
57461,2025.02.06 11:33:52.871,MF1,G42726444,finished,1847734,false,-2,[]


In [105]:
cols_with_mainLoop = [
    col 
    for col in dt_unknown.columns 
    if dt_unknown[col].astype(str).str.contains("Alert-Thread", na=False).any()
]
print(cols_with_mainLoop)

[]


In [126]:
unique_stages = dt_unknown['stage'].unique()
print(unique_stages)

['cleanMfsTrans' 'sendAvAlert_SND_LHM_EINGELAGERT' 'sendAVImpl'
 'sendRBGFS' 'updateFilled' 'updatePlanned' 'info' 'sendLhmFachtiefe'
 'checkCPU' 'mfsFachtiefe' 'sendRBGGB' 'sendLhmAusgelagert'
 'setStatusAnyWarte' 'sendFMDO' 'sendFMTA' 'cleanup' 'cleanMfsTransLog'
 'cleanRbgLog' 'cleanAuftrag' 'cleanSeqLog' 'cleanLogFm' 'cleanLogRbg'
 'cleanLogRbgEr' 'cleanLogRbgZs' 'cleanLogAv' 'cleanMfsTransErrTxt'
 'cleanBetriebsmittelLog' 'cleanMfsTransportStats' 'cleanNaviActive'
 'cleanLogRbgBck' 'cleanMfsTransLogBck' 'cleanAuftragsende' 'mfsLoeschen'
 'alert:MF1_FROMTSS' 'processAlertQueue'
 'sendAvAlert_SND_LHM_ANGEKOMMEN_LONG' 'mfsStart' 'mfsStart[laenge'
 'mfsStart[breite' 'mfsStart[hoehe' 'sendAvAlert_SND_LHM_ANGEKOMMEN_KURZ'
 'mfsEinlagern' 'mfsEinlagern[laenge' 'mfsEinlagern[breite'
 'mfsEinlagern[hoehe' 'sendFMLZ' 'updateLocks'
 'sendAvAlert_SND_LHM_VORBEIGEFAHREN' 'sendAvAlert_SND_LHM_ABGENOMMEN'
 'fa' 'getAndSetNioDestination' 'sendLhmWirdAusgeschleust' 'mfsAuslagern'
 'mfsAuslagern[pe

In [128]:
dt_start_handle_request.head()

,timestamp,module,worker_id,stage,request,area,le,force_check
0,2025.02.06 07:00:00.075,MF3,G42036921,START HANDLE REQUEST,CHECK_SEQ,MF2,1847044,true
1,2025.02.06 07:00:00.278,MF3,G42036931,START HANDLE REQUEST,CHECK_SEQ,MF2,1847105,true
2,2025.02.06 07:00:00.481,MF3,G42036941,START HANDLE REQUEST,CHECK_SEQ,MF2,1847079,true
3,2025.02.06 07:00:00.685,MF3,G42036950,START HANDLE REQUEST,CHECK_SEQ,MF2,1847015,true
4,2025.02.06 07:00:00.888,MF3,G42036961,START HANDLE REQUEST,CHECK_SEQ,MF2,1847053,true


In [116]:
dt_alerts_handled[dt_alerts_handled['timestamp'].isin(['2025.02.06 07:00:03.608', '2025.02.06 07:00:14.752', '2025.02.06 07:06:06.105', '2025.02.06 07:09:08.122', '2025.02.06 07:10:14.960', '2025.02.06 07:10:33.307'])]

,timestamp,module,worker_id,stage,alert_name,lfdnr,text,time_ms
0,2025.02.06 07:00:03.608,MF2,G42037093,Alert MF2_FROMTSS wurde verarbeitet,MF2_FROMTSS,None,2\18913\PG\0\PG\4\1208\1208\1\4\191\25\600\1\G...,0
1,2025.02.06 07:00:03.608,MF2,G42037094,Alert MF2_FROMTSS wurde verarbeitet,MF2_FROMTSS,None,3\21559\ST\3\--\0\0\0\0\0\0\0\0\ \ \ \0\0\0\0...,0
5,2025.02.06 07:00:14.752,MF2,G42037458,Alert MFS_FACHTIEFE_1 wurde verarbeitet,MFS_FACHTIEFE_1,None,MFS_ID1=1847164\HOST_LHM_ID1=10013880\Z_INFO_L...,0
74,2025.02.06 07:06:06.105,MF2,G42052143,Alert MF2_FROMTSS wurde verarbeitet,MF2_FROMTSS,None,2\18927\ST\3\--\0\0\0\0\0\0\0\0\ \ \ \0\0\0\0...,0
111,2025.02.06 07:09:08.122,MF1,G42060412,Alert MFS_START_1716 wurde verarbeitet,MFS_START_1716,None,MFS_ID1=1847087\HOST_LHM_ID1=10014460\Z_INFO_L...,0
154,2025.02.06 07:10:14.960,MF1,G42063571,Alert MFS_EINLAGERN_1054 wurde verarbeitet,MFS_EINLAGERN_1054,None,MFS_ID1=1847081\HOST_LHM_ID1=10034140\Z_INFO_L...,0
166,2025.02.06 07:10:33.307,MF1,G42064172,Alert MFS_START_1716 wurde verarbeitet,MFS_START_1716,None,MFS_ID1=1847095\HOST_LHM_ID1=10017744\Z_INFO_L...,0


In [118]:
dt_alerts_processing[dt_alerts_processing['timestamp'].isin(['2025.02.06 07:00:03.592', '2025.02.06 07:00:14.752', '2025.02.06 07:02:02.482', '2025.02.06 07:08:44.769', '2025.02.06 07:10:51.096', '2025.02.06 07:11:27.138'])]

,timestamp,module,worker_id,stage,alert_name,lfdnr,text,time_ms
0,2025.02.06 07:00:03.592,MF2,G42037093,Start Alert-Verarbeitung,MF2_FROMTSS,77809231,2\18913\PG\0\PG\4\1208\1208\1\4\191\25\600\1\G...,None
5,2025.02.06 07:00:14.752,MF2,G42037458,Start Alert-Verarbeitung,MFS_FACHTIEFE_1,77809240,MFS_ID1=1847164\HOST_LHM_ID1=10013880\Z_INFO_L...,None
33,2025.02.06 07:02:02.482,MF2,G42041498,Start Alert-Verarbeitung,MF2_FROMTSS,77809277,1\24777\ST\3\--\0\0\0\0\0\0\0\0\ \ \ \0\0\0\0...,None
103,2025.02.06 07:08:44.769,MF1,G42059431,Start Alert-Verarbeitung,MFS_LOESCHEN,77809347,HOST_LHM_ID1=10013851\MFS_ID1=1847021\Z_INFO_L...,None
181,2025.02.06 07:10:51.096,MF2,G42064949,Start Alert-Verarbeitung,MFS_FACHTIEFE_1,77809474,MFS_ID1=1847087\HOST_LHM_ID1=10014460\Z_INFO_L...,None
224,2025.02.06 07:11:27.138,MF1,G42067143,Start Alert-Verarbeitung,MF1_FROMTSS,77809548,101\18638\ST\0\101\0\0\7\0\0\0\0\0\0\xxxxxxxxx...,None


In [113]:
dt_alerts_received[dt_alerts_received['timestamp'].isin(['2025.02.06 07:11:13.574', '2025.02.06 07:11:16.856', '2025.02.06 07:11:17.121', '2025.02.06 07:11:17.981', '2025.02.06 07:11:17.981', ''])]

,timestamp,module,worker_id,stage,alert_name,lfdnr,text,time_ms
210,2025.02.06 07:11:13.574,Alert-Thread,G42066371,Alert empfangen,MF1_FROMTSS,None,1200\18630\ZZ\0\1200\1522\1207\0\2048\0\0\0\0\...,None
212,2025.02.06 07:11:16.856,Alert-Thread,G42066562,Alert empfangen,MF2_FROMTSS,None,3\21585\PG\0\PG\6\61\20\1200\5\1310\1310\1\1\G...,None
213,2025.02.06 07:11:17.121,Alert-Thread,G42066580,Alert empfangen,MFS_START_1726,None,MFS_ID1=1847174\HOST_LHM_ID1=10009235\Z_INFO_L...,None
216,2025.02.06 07:11:17.981,Alert-Thread,G42066635,Alert empfangen,MFS_EINLAGERN_1054,None,MFS_ID1=1847095\HOST_LHM_ID1=10017744\Z_INFO_L...,None
217,2025.02.06 07:11:17.981,Alert-Thread,G42066636,Alert empfangen,MFS_FACHTIEFE_1,None,MFS_ID1=1847081\HOST_LHM_ID1=10034140\Z_INFO_L...,None


In [121]:
dt_telegrams_processed

,timestamp,module,worker_id,stage,success,alert_name,text,tel_structure
0,2025.02.06 07:00:03.608,MF2,G42037093,TelegramDispatch processed,true,MF2_FROMTSS,2\18913\PG\0\PG\4\1208\1208\1\4\191\25\600\1\G...,RBG-AUFTRAG
1,2025.02.06 07:00:03.608,MF2,G42037094,TelegramDispatch processed,true,MF2_FROMTSS,3\21559\ST\3\--\0\0\0\0\0\0\0\0\ \ \ \0\0\0\0...,RBG-AUFTRAG
2,2025.02.06 07:00:03.623,MF2,G42037095,TelegramDispatch processed,true,MF2_FROMTSS,1\24771\ST\3\--\0\0\0\0\0\0\0\0\ \ \ \0\0\0\0...,RBG-AUFTRAG
3,2025.02.06 07:00:09.265,MF2,G42037201,TelegramDispatch processed,true,MF2_FROMTSS,1\24772\PF\0\PF\2\1108\1108\1\1\441\13\600\1\G...,RBG-AUFTRAG
4,2025.02.06 07:00:09.280,MF2,G42037209,TelegramDispatch processed,true,MF2_FROMTSS,4\354\ST\3\--\0\0\0\0\0\0\0\0\ \ \ \0\0\0\0\0...,RBG-AUFTRAG
...,...,...,...,...,...,...,...,...
7647,2025.02.06 11:59:30.956,MF2,G42762239,TelegramDispatch processed,true,MF2_FROMTSS,3\22610\ST\3\--\0\0\0\0\0\0\0\0\ \ \ \0\0\0\0...,RBG-AUFTRAG
7648,2025.02.06 11:59:30.956,MF2,G42762240,TelegramDispatch processed,true,MF2_FROMTSS,1\25711\ST\3\--\0\0\0\0\0\0\0\0\ \ \ \0\0\0\0...,RBG-AUFTRAG
7649,2025.02.06 11:59:30.956,MF2,G42762241,TelegramDispatch processed,true,MF2_FROMTSS,2\19949\ST\3\--\0\0\0\0\0\0\0\0\ \ \ \0\0\0\0...,RBG-AUFTRAG
7650,2025.02.06 11:59:30.972,MF2,G42762242,TelegramDispatch processed,true,MF2_FROMTSS,4\1139\ST\3\--\0\0\0\0\0\0\0\0\ \ \ \0\0\0\0\...,RBG-AUFTRAG


In [125]:
dt_telegrams_sent[dt_telegrams_sent['timestamp'].isin(['2025.02.06 07:00:03.608', '2025.02.06 07:00:03.951', ''])]

,timestamp,module,worker_id,stage,success,alert_name,text,tel_structure
0,2025.02.06 07:00:03.608,MF2,G42037093,Alert LHM_EINGELAGERT_1 wurde gesendet,None,LHM_EINGELAGERT_1,\MFS_ID1=1847066\HOST_LHM_ID1=10013097\LAGER1=...,None
1,2025.02.06 07:00:03.951,MF2,G42037098,Alert TSS_FROMMF2 wurde gesendet,None,TSS_FROMMF2,2\0\EA\0\FS\3\281\3\1200\3\1210\1210\1\1\G\XX\...,None
2,2025.02.06 07:00:09.265,MF2,G42037201,Alert LHM_FACHTIEFE_1 wurde gesendet,None,LHM_FACHTIEFE_1,\MFS_ID1=1847164\HOST_LHM_ID1=10013880\LAGER1=...,None
3,2025.02.06 07:00:14.862,MF2,G42037470,Alert TSS_FROMMF2 wurde gesendet,None,TSS_FROMMF2,1\0\EA\0\GB\2\1108\1108\1\1\441\13\600\1\G\XX\...,None
4,2025.02.06 07:00:31.043,MF2,G42038074,Alert LHM_AUSGELAGERT_1 wurde gesendet,None,LHM_AUSGELAGERT_1,\MFS_ID1=1847145\HOST_LHM_ID1=10018335\Z_INFO_...,None
...,...,...,...,...,...,...,...,...
5927,2025.02.06 11:34:05.341,MF1,G42726756,Alert TSS_FROMMF1 wurde gesendet,None,TSS_FROMMF1,1407\0\TA\0\1407\1407\1408\0\1024\0\104007\100...,None
5928,2025.02.06 11:34:43.640,MF2,G42726678,Alert TSS_FROMMF2 wurde gesendet,None,TSS_FROMMF2,4\0\EA\0\FS\8\1408\1408\1\7\51\15\600\1\G\XX\0...,None
5929,2025.02.06 11:35:21.782,MF2,G42728559,Alert LHM_FACHTIEFE_1 wurde gesendet,None,LHM_FACHTIEFE_1,\MFS_ID1=1847796\HOST_LHM_ID1=10013924\LAGER1=...,None
5930,2025.02.06 11:36:00.289,MF2,G42728574,Alert TSS_FROMMF2 wurde gesendet,None,TSS_FROMMF2,4\0\EA\0\GB\8\1408\1408\1\7\51\15\600\1\G\XX\0...,None
